In [1]:
%pylab qt5
from unittest.mock import MagicMock, Mock
from time import sleep
class DataGen:
    
    def __init__(self):
        self._counter = 0
        self._nop = 1
        
    def fun(self, x):
        return [cos(0.05*x) + 1j*cos(0.05*x)]*self._nop
     #   return 33
    def get_sdata(self):
        retval = self.fun(self._counter)
        self._counter += 1
        sleep(0.1)
        return retval
    
    def get_frequencies(self):
        return linspace(self._freq_limits[0], self._freq_limits[1], self._nop)
    
    def set_parameters(self, params):
        self._nop = params["nop"]
        self._freq_limits = params["freq_limits"]
    
    def set_freq_limits(self, *limits):
        self._freq_limits = limits
        
    def set_nop(self, nop):
        self._nop = nop
        
    def detect(self):
        return 5.9e9, 0, 0


    
dg = DataGen()
vna = Mock()
cur_src = Mock()
q_lo = Mock()
q_awg = Mock()
ro_awg = MagicMock()
q_lo_2 = Mock()
q_awg_2 = Mock()
ro_awg_2 = MagicMock()
mw_src = Mock()
RD = Mock()
RD.detect = dg.detect
vna.get_sdata = dg.get_sdata
vna.set_parameters = dg.set_parameters
vna.get_frequencies = dg.get_frequencies
vna.set_freq_limits = dg.set_freq_limits
vna.set_nop = dg.set_nop


Populating the interactive namespace from numpy and matplotlib


# Echo

In [2]:
from lib2.DispersiveHahnEcho import *
DHE = DispersiveHahnEcho("hahn-echo", 'lol',
                         vna=[vna], q_lo=[q_lo], q_awg=[q_awg], ro_awg=[ro_awg])
vna_parameters= {"bandwidth":10, "freq_limits":[6e9]*2, "nop":10, "averages":1}
q_freq = 5e9
echo_delays = linspace(0, 20000, 201)
sequence_parameters = {"awg_trigger_reaction_delay":0, "readout_duration":3000, 
                            "repetition_period":30000, "half_pi_pulse_duration":100/2}
ro_awg_params =  {"calibration":MagicMock()}
q_awg_params = {"calibration":MagicMock()}
q_lo_params = {'power':-10, 'frequency':q_freq + 100e6}
dev_params = {'vna':[vna_parameters], 
              'ro_awg':[ro_awg_params], 
              'q_awg':[q_awg_params],
              'q_lo':[q_lo_params]}
DHE.set_fixed_parameters(sequence_parameters, **dev_params)
DHE.set_swept_parameters(echo_delays)
# DHE.set_ult_calib(True)
# DHE.set_basis(basis)

In [3]:
dhe_result = DHE.launch()

Started at:  2018-11-21 17:08:05.858142
Time left: 0 h 0 m 14.42 s, [echo_delay: 6.90e+03], average cycle time: 0.11 s       

C:\Users\pogor\YandexDisk\Ustinov internship\Measurement-automation\lib2\VNATimeResolvedDispersiveMeasurement1D.py:81: RuntimeWarning: invalid value encountered in sqrt
  return result, sqrt(diag(sigma ** 2 * inv(result.jac.T.dot(result.jac))))


Time left: 0 h 0 m 0.0 s, [echo_delay: 2.00e+04], average cycle time: 0.11 s       
Elapsed time: 0 h 0 m 22.25 s


# Decay

In [3]:
from lib2.DispersiveDecay import *
DD = DispersiveDecay("decay", 'lol',
                     vna=[vna], q_lo=[q_lo], q_awg=[q_awg], ro_awg=[ro_awg])
vna_parameters= {"bandwidth":10, "freq_limits":[6e9]*2, "nop":10, "averages":1}
q_freq = 5e9
readout_delays = linspace(0, 20000, 51)
decay_sequence_parameters = {"awg_trigger_reaction_delay":0, "readout_duration":3000, 
                            "repetition_period":30000, "pi_pulse_duration":100}
ro_awg_params =  {"calibration":MagicMock()}
q_awg_params = {"calibration":MagicMock()}
q_lo_params = {'power':-10, 'frequency':q_freq + 100e6}
dev_params = {'vna':[vna_parameters], 
              'ro_awg':[ro_awg_params], 
              'q_awg':[q_awg_params],
              'q_lo':[q_lo_params]}
DD.set_fixed_parameters(decay_sequence_parameters, **dev_params)
DD.set_swept_parameters(readout_delays)
#DD.set_basis(basis)

In [4]:
dd_result = DD.launch()

Started at:  2018-11-20 18:03:04.460196
Time left: 0 h 0 m 2.27 s, [readout_delay: 1.56e+04], average cycle time: 0.21 s       

C:\Users\pogor\YandexDisk\Ustinov internship\Measurement-automation\lib2\VNATimeResolvedDispersiveMeasurement1D.py:81: RuntimeWarning: invalid value encountered in sqrt
  return result, sqrt(diag(sigma**2*inv(result.jac.T.dot(result.jac))))


Time left: 0 h 0 m 0.0 s, [readout_delay: 2.00e+04], average cycle time: 0.21 s       
Elapsed time: 0 h 0 m 10.51 s


In [878]:
dd_result.visualize();

# Ramsey

In [2]:
from lib2.DispersiveRamsey import *
DR = DispersiveRamsey("ramsey", "lol", 
                      vna=[vna], q_lo=[q_lo], q_awg=[q_awg], ro_awg=[ro_awg])
vna_parameters= {"bandwidth":10, "freq_limits":[6e9]*2, "nop":10, "averages":1}
ramsey_delays = linspace(0, 10000, 201)
q_freq = 5e9
exc_frequency = q_freq - 1e6
pulse_sequence_parameters = {"awg_trigger_reaction_delay":0, "readout_duration":3000, 
                                "repetition_period":15000, "half_pi_pulse_duration":100/2}

ro_awg_params =  {"calibration":MagicMock()}
q_awg_params = {"calibration":MagicMock()}
q_lo_params = {'power':-10, 'frequency':exc_frequency + 100e6}
dev_params = {'vna':[vna_parameters], 
              'ro_awg':[ro_awg_params], 
              'q_awg':[q_awg_params],
              'q_lo':[q_lo_params]}

DR.set_fixed_parameters(pulse_sequence_parameters, **dev_params)
DR.set_swept_parameters(ramsey_delays)
DR.set_basis((1 + 1j,-1 - 1j))
# DR.set_ult_calib(True)

In [3]:
dr_result = DR.launch()

Started at:  2018-11-21 16:53:19.454638
Time left: 0 h 0 m 21.47 s, [ramsey_delay: 5.50e+02], average cycle time: 0.11 s       

C:\Users\pogor\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\optimize\minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


Time left: 0 h 0 m 0.0 s, [ramsey_delay: 1.00e+04], average cycle time: 0.11 s       
Elapsed time: 0 h 0 m 21.41 s


# TTS

In [2]:
from lib2.TwoToneSpectroscopy import *
TTS = FluxTwoToneSpectroscopy('TTS', 'lol', 
                              vna =[vna], mw_src=[mw_src], current_src=[cur_src])
TTS._resonator_detector = RD
res_limits = [4e9, 6e9]
vna_parameters = {"bandwidth":250, "freq_limits":res_limits, "nop":10, "power":-15, "averages":1}
mw_src_parameters = {"power":0}
mw_src_frequencies = linspace(8.5e9, 9.2e9, 51)
center = 0
sweet_spot = 0
period = 1
currents = linspace(sweet_spot - period/8, sweet_spot + period/8, 51) # comment out to use fluxes from STS
#voltages = linspace(0, 0.9, 51) # comment out to use voltages from STS
#sweet_spot_current = None
dev_params = {'vna': [vna_parameters],
              'mw_src': [mw_src_parameters]}
TTS.set_fixed_parameters(sweet_spot_current=mean(currents), adaptive=False, **dev_params)
#TTS.set_swept_parameters(mw_src_frequencies, voltage_values = voltages)
TTS.set_swept_parameters(mw_src_frequencies, current_values = currents)
TTS._measurement_result._unwrap_phase = True

Detecting a resonator within provided frequency range of the VNA [4000000000.0, 6000000000.0]                             at 0.00 mA
Detected frequency is 5.90000 GHz, at 0.00 mU and 0.00 degrees


In [ ]:
tts_result = TTS.launch()

Started at:  2018-11-19 12:28:55.079793
Time left: 0 h 7 m 7.56 s, [Current [A]: -8.00e-02, Frequency [Hz]: 8.81e+09], average cycle time: 0.2 s       

# STS

In [4]:
from lib2.SingleToneSpectroscopy import *
currents = linspace(-.75e-4, .75e-4, 101)
voltages = linspace(-.75,1, 101)
res_limits = [4e9, 6e9]

STS = SingleToneSpectroscopy('STS', 'lol', plot_update_interval = 1, vna = [vna], src = [cur_src])
vna_parameters = {"bandwidth":500, "freq_limits":res_limits, "nop":101, "power":-20, "averages":1}
dev_params = {'vna': [vna_parameters]}

STS.set_fixed_parameters(**dev_params)
STS._src[0].set_current(currents[0])
sleep(1)
STS.set_swept_parameters({'Current [A]': (STS._src[0].set_current, currents)})
# STS.set_swept_parameters({'AWG voltage [V]': (voltage_awg.set_voltage, voltages)})

sts_result = STS.launch()

Started at:  2018-11-21 16:59:45.332811
Time left: 0 h 0 m 0.0 s, [Current [A]: 7.50e-05], average cycle time: 0.1 s       
Elapsed time: 0 h 0 m 10.44 s


In [3]:
STS.stop()

Measurement was interrupted! 

Time left: 0 h 1 m 6.54 s, [Current [A]: -1.77e-05], average cycle time: 0.11 s       

In [5]:
sts_result.visualize()

(<Figure size 1500x700 with 4 Axes>,
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x00000239EC0D05F8>,
       dtype=object),
 (<matplotlib.axes._axes.Axes at 0x23980134358>,
  <matplotlib.axes._axes.Axes at 0x23980598f28>))

In [21]:
plt.plot([1,2,3],[1,2])

ValueError: x and y must have same first dimension, but have shapes (3,) and (2,)

In [15]:
from lib2.DispersiveRabiOscillations import *

DRO = DispersiveRabiOscillations("rabi", "lol", plot_update_interval=0.5,
            vna=[vna], q_lo=[q_lo], q_awg=[q_awg], ro_awg=[ro_awg])
vna_parameters = {"bandwidth":10, "freq_limits":[5e9]*2, "nop":10, "averages":1}
exc_frequency = 6e9
excitation_durations = linspace(0, 2000, 201)

rabi_sequence_parameters = pulse_sequence_parameters= {"awg_trigger_reaction_delay":0, "excitation_amplitude":1,
                                                "readout_duration":3000, "repetition_period":15000}
ro_awg_params =  {"calibration":MagicMock()}
q_awg_params = {"calibration":MagicMock()}
q_lo_params = {'power':-10, 'frequency':exc_frequency + 100e6}
dev_params = {'vna':[vna_parameters], 
              'ro_awg':[ro_awg_params], 
              'q_awg':[q_awg_params],
              'q_lo':[q_lo_params]}
DRO.set_fixed_parameters(rabi_sequence_parameters, **dev_params)
DRO.set_swept_parameters(excitation_durations)
#DRO.set_basis(basis)
DRO.set_ult_calib(False)

In [16]:
result = DRO.launch()

Started at:  2018-11-20 17:46:46.153761
Time left: 0 h 0 m 39.97 s, [excitation_duration: 6.00e+01], average cycle time: 0.21 s       

C:\Users\pogor\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\optimize\minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


Time left: 0 h 0 m 0.0 s, [excitation_duration: 2.00e+03], average cycle time: 0.21 s       
Elapsed time: 0 h 0 m 41.53 s


In [18]:
basis = result.get_basis()

# Joint Rabi

In [6]:
from lib2.DispersiveRabiOscillationsJoint import *

DROJ = DispersiveRabiOscillationsJoint("rabi", "lol",
            vna=[vna], q_lo=[q_lo, q_lo_2], q_awg=[q_awg, q_awg_2], ro_awg=[ro_awg], plot_update_interval=0.5)
vna_parameters = {"bandwidth":10, "freq_limits":[5e9]*2, "nop":10, "averages":1}
exc_frequency = 6e9
excitation_durations = linspace(0, 2000, 201)

rabi_sequence_parameters = pulse_sequence_parameters= {"awg_trigger_reaction_delay":0, "excitation_amplitude":1,
                                                "readout_duration":3000, "repetition_period":15000}
ro_awg_params =  {"calibration":MagicMock()}
q_awg_params = {"calibration":MagicMock()}
q_lo_params = {'power':-10, 'frequency':exc_frequency + 100e6}
dev_params = {'vna':[vna_parameters], 
              'ro_awg':[ro_awg_params], 
              'q_awg':[q_awg_params, q_awg_params],
              'q_lo':[q_lo_params, q_lo_params]}
DROJ.set_fixed_parameters(rabi_sequence_parameters, **dev_params)
DROJ.set_swept_parameters(excitation_durations)
#DRO.set_basis(basis)
DROJ.set_ult_calib(False)

In [7]:
DROJ.launch()

Started at:  2018-11-16 16:03:58.127108
Time left: 0 h 0 m 42.96 s, [excitation_duration: 7.00e+01], average cycle time: 0.22 s       

C:\Users\pogor\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\optimize\minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\pogor\YandexDisk\Ustinov internship\Measurement-automation\lib2\VNATimeResolvedDispersiveMeasurement1D.py:81: RuntimeWarning: invalid value encountered in sqrt
  return result, sqrt(diag(sigma**2*inv(result.jac.T.dot(result.jac))))


Time left: 0 h 0 m 0.0 s, [excitation_duration: 2.00e+03], average cycle time: 0.22 s       
Elapsed time: 0 h 0 m 45.12 s
